# First Steps with GMT/Python


This tutorial will get you started with the basic usage of GMT/Python.
Some of the examples shown here are from the [GMT Tutorial](http://gmt.soest.hawaii.edu/doc/latest/GMT_Tutorial.html#session-one).

## Loading the library

The GMT modules are available as functions and classes in the `gmt` Python package. 
So we'll start by importing it:

In [ ]:
import gmt

## Our first map

All figure generation in GMT/Python is handled by the `gmt.Figure` class. 
It has methods to add layers to your figure, like a basemap, coastlines, and data.

We start a new figure by creating an instance of `gmt.Figure`:

In [ ]:
fig = gmt.Figure()

We add elements to the figure using its methods. For example, lets add the coastlines of Central America to a 6 inch wide map using the Mercator projection (`M`). Our figure will also have a nice frame with automatic ticks.

In [ ]:
fig.coast(region=[-90, -70, 0, 20], projection='M6i', land='chocolate', 
          frame=True)

You can see a preview of the figure directly in the [Jupyter notebook](http://jupyter.org) using `fig.show()`.

In [ ]:
fig.show()

To open a PDF preview of the figure using your default PDF viewer use:

```python
fig.show(method='external')
```

This is useful when using the Python shell or IPython terminal app. 
However, **this command will not interrupt your Python process**. 
So using it in a Python script will not work because the script will finish and delete the generated previews.
Use `fig.savefig` to save the figure to a file instead (see below).

There is also the option of inserting the figure in an **interactive globe** using [NASA's WorldWind Web](https://worldwind.arc.nasa.gov/web/). See option `external='globe'` in the examples below.

## A note for experienced GMT users

You'll probably have noticed several things that are different from classic command-line GMT.
Many of these changes reflect the new GMT [modern execution mode](http://gmt.soest.hawaii.edu/projects/gmt/wiki/Modernization) that will be part of the future 6.0 release.
A few are GMT/Python exclusive (like the long argument names).

1. The name of method is `coast` instead of `pscoast`. As a general rule, all `ps*` modules had their `ps` removed. The exceptions are: `psxy == plot`, `psxyz == plot3d`, and `psscale == colorbar`.
2. The arguments don't use the GMT 1-letter syntax (R, J, B, etc). Those are still available as aliases and the methods will accept them (see below). 
3. Arguments like `region` can take lists instead of strings like `1/2/3/4`. You can still use the string form but the list form is easier in Python.
4. If a GMT argument has no options (like `-B` instead of `-Baf`), use a `True` value instead. An empty string would also be acceptable.
5. There is no output redirecting to a PostScript file. The figure is generated in the background and will only be shown or saved when you ask for it.

We could have generated the figure above using the classic GMT argument names (but not the module names):

In [ ]:
fig_alias = gmt.Figure()
fig_alias.coast(R='-90/-70/0/20', J='M6i', G='gray', S="blue", B=True)
fig_alias.show()

## Saving the figure

Unlike the GMT command-line interface, **no figure file was generated until you ask for one**.  
That means that `fig.show` won't produce a figure file.

Use method `fig.savefig` (based on the [matplotlib](https://matplotlib.org/) function) to save your figure:

In [ ]:
fig.savefig('first-steps-central-america.png')

If you're running a Python script, you can tell `fig.savefig` to open the figure in an external viewer:

```python
fig.savefig('first-steps-central-america.png', show=True)
```

## Cartesian plots

The `gmt.Figure` class has a `plot` method for displaying points and lines. Let's make a Cartesian x, y plot using some random data generated using numpy:

In [ ]:
import numpy as np

In [ ]:
# See the random number generator so we always 
# get the same numbers
np.random.seed(42)
ndata = 100
region = [150, 240, -10, 60]
# Create some fake distribution of points and a measured value
x = np.random.uniform(region[0], region[1], ndata)
y = np.random.uniform(region[2], region[3], ndata)
magnitude = np.random.uniform(1, 5, size=ndata)

Now we can plot the data using `Figure.plot` and the Cartesian projection `X`.

In [ ]:
fig = gmt.Figure()
# Create a 6x6 inch basemap using the data region
fig.basemap(region=region, projection='X6i', frame=True)
# Plot using triangles (i) of 0.3 cm
fig.plot(x, y, style='i0.3c', color='black')
fig.show()

We can make the size of the markers follow the fake "magnitude" values by passing in the argument `sizes` to `Figure.plot`. We'll need to scale the magnitude so that it will reflect the size in centimeters.

In [ ]:
fig = gmt.Figure()
fig.basemap(region=region, projection='X6i', frame=True)
fig.plot(x, y, style='ic', color='black', sizes=magnitude/10)
fig.show()

## Plotting directly from a file

Sometimes you'll have data in a file that you just want to plot without having to load it into Python. You can use the `data` argument of `Figure.plot` to specify the file name instead `x` and `y`. GMT will take care of loading your data and plotting.

In [ ]:
# Save our fake data to a file.
np.savetxt('first-steps-data.txt', 
           np.transpose([x, y, magnitude]))

The `columns` argument controls which columns are used as x, y, color, and size, respectively. GMT allows some basic operations on the column values before plotting. For example, adding `sVALUE` to a column index will multiply it by that value before plotting.

In [ ]:
fig = gmt.Figure()
fig.basemap(region=region, projection='X6i', frame=True)
fig.plot(data='first-steps-data.txt', style='cc', color='red', 
         columns=[0, 1, '2s0.1'])
fig.show()

## Making maps using sample data

GMT shines when plotting data on a map. We can use some **sample data** that is packaged with GMT to try this out. They can be accessed using special file names that begin with an `@` symbol, for example `@tut_quakes.ngdc`. You can supply these names as the `data` argument in `Figure.plot` and other plotting functions. If you don't have the files already, they are automatically downloaded by GMT and saved to a cache directory (usually `~/.gmt/cache`).

The `gmt.datasets` package allows easy access to these data files as Python data types. For example, we can access the sample dataset of tsunami generating earthquakes around Japan (`@tut_quakes.ngdc`) as a `pandas.DataFrame` using the `datasets.load_japan_quakes` function:

In [ ]:
from gmt.datasets import load_japan_quakes

quakes = load_japan_quakes()
quakes.head()

The functions returns to us the data in a `pandas.Dataframe` object that contains the date, hypocenter coordinates, and magnitude of the earthquakes.

Let's make a local Mercator map of the epicenter coordinates. 

In [ ]:
quakes_region = [quakes.longitude.min() - 1, quakes.longitude.max() + 1,
                 quakes.latitude.min() - 1, quakes.latitude.max() + 1]

In [ ]:
fig = gmt.Figure()
fig.coast(region=quakes_region, projection='M6i', frame=True, 
          land='black', water='skyblue')
fig.plot(x=quakes.longitude, y=quakes.latitude, 
         style='c0.3c', color='white', pen='black')
fig.show()

As before, we can map the size of the markers to the earthquake magnitude. Because the magnitude is on a logarithmic scale, it helps to show the differences by scaling the values using a power law.

In [ ]:
fig = gmt.Figure()
fig.coast(region=quakes_region, projection='M6i', frame=True, 
          land='black', water='skyblue')
fig.plot(x=quakes.longitude, y=quakes.latitude, 
         sizes=0.02*(2**quakes.magnitude),
         style='cc', color='white', pen='black')
fig.show()

We can also map the colors of the markers to the depths by passing an array to the `color` argument and providing a colormap name (`cmap`). We can even use the new matplotlib colormap `"viridis"`.

In [ ]:
fig = gmt.Figure()
fig.coast(region=quakes_region, projection='M6i', frame=True, 
          land='black', water='skyblue')
fig.plot(x=quakes.longitude, y=quakes.latitude, 
         sizes=0.02*2**quakes.magnitude,
         color=quakes.depth_km/quakes.depth_km.max(),
         cmap='viridis', style='cc', pen='black')
fig.show()

Let's preview this map using the **interactive globe**. In this case, we don't need the frame or color in the oceans. We must also use a **Cartesian projection** (X) and degrees (d) for plot units so that the figure can be aligned with the globe.

In [ ]:
fig = gmt.Figure()
fig.coast(region=quakes_region, projection='X6id/6id', land='gray')
fig.plot(x=quakes.longitude, y=quakes.latitude, 
         sizes=0.02*2**quakes.magnitude,
         color=quakes.depth_km/quakes.depth_km.max(),
         cmap='viridis', style='cc', pen='black')
fig.show(method='globe')